# **Prueba de uso *Open-Meteo Historical Weather API***

Variables disponibles en Open-Meteo Historical Weather API:
1. Temperaturas:

    - ``temperature_2m_min``: Temperatura mínima a 2 metros.

    - ``temperature_2m_max``: Temperatura máxima a 2 metros.

    - ``temperature_2m_mean``: Temperatura media a 2 metros (promedio entre mínima y máxima).

2. Precipitación:

    - ``precipitation_sum``: Precipitación total en el día (en milímetros).

    - ``precipitation_hours``: Horas de lluvia en el día.

3. Humedad:

    - ``humidity_2m_mean``: Humedad relativa media a 2 metros.

    - ``humidity_2m_max``: Humedad máxima a 2 metros.

    - ``humidity_2m_min``: Humedad mínima a 2 metros.

4. Viento:

    - ``windspeed_10m_max``: Velocidad máxima del viento a 10 metros.

    - ``windspeed_10m_mean``: Velocidad media del viento a 10 metros.

    - ``windgusts_10m_max``: Ráfagas máximas del viento a 10 metros.

5. Radiación solar:

    - ``radiation_sum``: Radiación solar total en el día (en MJ/m²).

6. Cobertura de nubes:

    - ``cloudcover_mean``: Promedio de la cobertura de nubes (en porcentaje).

    - ``cloudcover_max``: Cobertura máxima de nubes (en porcentaje).

    - ``cloudcover_min``: Cobertura mínima de nubes (en porcentaje).

7. Otros:

    - ``pressure_mean``: Presión atmosférica media.

    - ``weathercode``: Código del tipo de tiempo (despejado, lluvioso, etc.).

params = {
    "start_date": "2018-01-01",
    "end_date": "2018-12-31",
    "daily": "temperature_2m_max,precipitation_sum,humidity_2m_mean",
    "timezone": "America/Bogota"
}

TEMPERATURA, HUMEDAD Y VIENTO

Open-Meteo Historical Weather API

https://open-meteo.com/en/docs/historical-weather-api

In [3]:
import pandas as pd
import requests

In [15]:
# Coordenadas de las capitales de la región Caribe
capitales = {
    'Barranquilla': {'lat': 10.9878, 'lon': -74.7889},
    'Cartagena': {'lat': 10.3910, 'lon': -75.4794},
    'Valledupar': {'lat': 10.4639, 'lon': -73.2505},
    'Riohacha': {'lat': 11.5441, 'lon': -72.9074},
    'Santa Marta': {'lat': 11.2408, 'lon': -74.1990},
    'San Andrés': {'lat': 12.5840, 'lon': -81.7117},
    'Sincelejo': {'lat': 9.3037, 'lon': -75.3972},
    'Montería': {'lat': 8.7491, 'lon': -75.8813}
}

# Parámetros comunes para la API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "start_date": "2018-01-01",
    "end_date": "2018-12-31",
    "daily": "temperature_2m_max",
    "timezone": "America/Bogota"
}

# Diccionario para almacenar los resultados
resultados = {}

# Peticiones a la API para cada capital
for ciudad, coords in capitales.items():
    params['latitude'] = coords['lat']
    params['longitude'] = coords['lon']
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # Convertir los datos a DataFrame y calcular el promedio mensual
    df = pd.DataFrame({
        'fecha': data['daily']['time'],
        'temp_max': data['daily']['temperature_2m_max']
    })
    df['fecha'] = pd.to_datetime(df['fecha'])
    df.set_index('fecha', inplace=True)
    mensual_max = df.resample('ME').mean()
    
    # Guardar los resultados
    resultados[ciudad] = mensual_max


In [16]:
resultados['Barranquilla']

,temp_max
fecha,
2018-01-31,29.622581
2018-02-28,28.935714
2018-03-31,29.783871
2018-04-30,30.570000
2018-05-31,30.016129
2018-06-30,30.500000
2018-07-31,30.361290
2018-08-31,30.745161
2018-09-30,30.606667


# **Prueba de uso *Open-Meteo Air Quality API***

In [7]:
import requests
import pandas as pd

capitales = {
    'Barranquilla': {'lat': 10.9878, 'lon': -74.7889},
    'Cartagena': {'lat': 10.3910, 'lon': -75.4794},
    'Valledupar': {'lat': 10.4639, 'lon': -73.2505},
    'Riohacha': {'lat': 11.5441, 'lon': -72.9074},
    'Santa Marta': {'lat': 11.2408, 'lon': -74.1990},
    'San Andrés': {'lat': 12.5840, 'lon': -81.7117},
    'Sincelejo': {'lat': 9.3037, 'lon': -75.3972},
    'Montería': {'lat': 8.7491, 'lon': -75.8813}
}

url = "https://air-quality-api.open-meteo.com/v1/air-quality"

params_base = {
    "start_date": "2023-01-02",
    "end_date": "2024-01-02",
    "daily": "pm10",
    "timezone": "America/Bogota"
}

resultados = {}

for ciudad, coords in capitales.items():
    params = params_base.copy()
    params['latitude'] = coords['lat']
    params['longitude'] = coords['lon']
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        pm10_values = data.get("daily", {}).get("pm10", [])
        fechas = data.get("daily", {}).get("time", [])
        
        if pm10_values and fechas:
            df = pd.DataFrame({
                'fecha': pd.to_datetime(fechas),
                'pm10': pm10_values
            })
            df.set_index('fecha', inplace=True)
            mensual_pm10 = df.resample('M').mean()
            resultados[ciudad] = mensual_pm10
        else:
            print(f"No hay datos para {ciudad}")
    else:
        print(f"Error en la consulta para {ciudad}: {response.status_code}")

# Mostrar promedio mensual PM10 en Barranquilla
print(resultados.get('Barranquilla'))


No hay datos para Barranquilla
No hay datos para Cartagena
No hay datos para Valledupar
No hay datos para Riohacha
No hay datos para Santa Marta
No hay datos para San Andrés
No hay datos para Sincelejo
No hay datos para Montería
None
